In [497]:
import pandas as pd
import glob
import os
from sklearn.preprocessing import LabelEncoder

files = os.path.join("outlierr", "*.csv")
files = glob.glob(files)
df1 = pd.concat(map(pd.read_csv, files), ignore_index=True)
df1.to_csv('divar_out_remove.csv')
df = pd.read_csv('divar_out_remove.csv', encoding="utf-8")


In [498]:
import numpy as np
import pandas as pd
from unidecode import unidecode
import re


In [499]:
df.shape

(33828, 20)

In [500]:
df.head()

,Unnamed: 0,Unnamed: 0.1,slug,category_title,city,district,latin_city,meter,built_year,total_cost,price_per_meter,floor,room,elevator,parking,storage,divder,lat,long,token
0,0,0,apartment-sell,آپارتمان,تهران,اسکندری,tehran,۱۳۰,۱۴۰۰,توافقی,NaN,۳,۳,1.0,1.0,1.0,1.0,35.692722,51.382103,gYzd0pQh
1,1,1,apartment-sell,آپارتمان,تهران,اسکندری,tehran,۸۳,۱۳۸۹,۲٬۲۰۰٬۰۰۰٬۰۰۰,26506000.0,۲ از ۴,۲,NaN,NaN,1.0,1.0,35.686893,51.381820,gYAyPE6u
2,2,2,apartment-sell,آپارتمان,تهران,اسکندری,tehran,۴۹,۱۳۸۰,۱٬۱۴۰٬۰۰۰٬۰۰۰,23265000.0,۱,۱,NaN,NaN,1.0,1.0,NaN,NaN,gY9Fxf39
3,3,3,apartment-sell,آپارتمان,تهران,اسکندری,tehran,۵۰,۱۳۸۳,۱٬۵۰۰٬۰۰۰٬۰۰۰,30000000.0,۳,۱,NaN,1.0,1.0,1.0,NaN,NaN,gYAqfK5w
4,4,4,apartment-sell,آپارتمان,تهران,اسکندری,tehran,۵۳,۱۳۷۸,۱٬۴۰۰٬۰۰۰٬۰۰۰,26415000.0,۲ از ۴,۱,NaN,NaN,NaN,1.0,35.693096,51.382053,gY4tLvuX


In [501]:
def ConvertNumbers(persian_int):
        return unidecode(f"{persian_int}")

df['floor_en'] = df['floor'].apply(ConvertNumbers)
df.drop(df[df['floor_en']=='nan'].index, inplace=True)


def tabaghe_t(x):
    result=None
    floor=x.split()
    if 'hmkhf' in x:
        result=('0',floor[-1])
    elif 'zyrhmkhf' in x:
        result=('-1',floor[-1])
    elif 'byshtr' in x:
        result=(floor[0],floor[-1])
    else:
        result= re.findall ('\d+',x)
    return result

def total_floor(x):
    try:
        return x[1]
    except:
        return x[0]

floors=df.floor_en.apply(tabaghe_t)
df['floor_en']=floors.apply(lambda x: x[0])
df['total_floor']=floors.apply(total_floor)
df.total_floor.replace(to_replace='hmkhf', value='0', inplace=True)
df.total_floor.replace(to_replace='zyrhmkhf', value='-1', inplace=True)

In [502]:
df.drop(['floor'], axis =1, inplace =True)

In [503]:
df=df[df.total_cost != 'توافقی']


In [504]:
df.loc[:,'total_cost']= df['total_cost'].astype(str).str.replace("تومان", "")
df.loc[:,'price_per_meter'] = df['price_per_meter'].astype(str).str.replace("تومان", "")


In [505]:
df.loc[:,'elevator']=df['elevator'].fillna(0)
df.loc[:,'parking']=df['parking'].fillna(0)
df.loc[:,'storage']=df['storage'].fillna(0)

In [506]:
df.drop(['lat','long','Unnamed: 0.1','Unnamed: 0','slug','category_title','latin_city','divder','total_cost','city','token'], axis =1, inplace =True)

In [507]:
df.drop(df[df['floor_en']=='30+'].index,axis =0,inplace=True)


In [508]:

# df.loc[:,'floor_en'] = pd.to_numeric(df.loc[:,'floor_en'].astype(str).str.replace("30+", "30"))
df.loc[:,'room'] = df.loc[:,'room'].astype(str).str.replace("بدون اتاق", "0")
df['room'] = df['room'].apply(ConvertNumbers)
df['built_year'] = df['built_year'].apply(ConvertNumbers)
df.loc[:,'built_year'] = pd.to_numeric(df.loc[:,'built_year'].astype(str).str.replace("qbl z", ""))
df.loc[:,'room'] = pd.to_numeric(df.loc[:,'room'].astype(str).str.replace("4+", "5"))


/tmp/ipykernel_31756/2762238869.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:,'room'] = pd.to_numeric(df.loc[:,'room'].astype(str).str.replace("4+", "5"))


In [509]:

num = [1,3,8,9]
for i in num:
    df.loc[:,df.columns[i]] = pd.to_numeric(df[df.columns[i]].astype(str).str.replace("٬", "").apply(unidecode), errors='coerce')
   

In [510]:
df.loc[:,'room']=df['room'].fillna('5')
df.dropna(subset=['district'],inplace = True, axis=0)
df.dropna(subset=['room'],inplace = True, axis=0)

In [511]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25297 entries, 1 to 33827
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   district         25297 non-null  object 
 1   meter            25297 non-null  int64  
 2   built_year       25297 non-null  int64  
 3   price_per_meter  25297 non-null  float64
 4   room             25297 non-null  int64  
 5   elevator         25297 non-null  float64
 6   parking          25297 non-null  float64
 7   storage          25297 non-null  float64
 8   floor_en         25297 non-null  int64  
 9   total_floor      25297 non-null  int64  
dtypes: float64(4), int64(5), object(1)
memory usage: 2.1+ MB


In [512]:
le = LabelEncoder()
label=le.fit_transform(df['district'])
df['district_code']=label

In [513]:
df

,district,meter,built_year,price_per_meter,room,elevator,parking,storage,floor_en,total_floor,district_code
1,اسکندری,83,1389,26506000.0,2,0.0,0.0,1.0,2,4,25
2,اسکندری,49,1380,23265000.0,1,0.0,0.0,1.0,1,1,25
3,اسکندری,50,1383,30000000.0,1,0.0,1.0,1.0,3,3,25
4,اسکندری,53,1378,26415000.0,1,0.0,0.0,0.0,2,4,25
5,اسکندری,42,1400,40000000.0,1,1.0,0.0,1.0,3,4,25
...,...,...,...,...,...,...,...,...,...,...,...
33823,المهدی,68,1382,32353000.0,1,0.0,0.0,1.0,3,3,28
33824,المهدی,70,1387,41429000.0,2,1.0,1.0,1.0,2,2,28
33825,المهدی,102,1390,47059000.0,2,1.0,1.0,1.0,5,5,28
33826,المهدی,131,1388,44962000.0,3,0.0,1.0,1.0,1,1,28


In [ ]:
df.to_csv("")